# GenParse: Lark Interface

In [1]:
import lark

from genparse.util import LarkStuff
from arsenal import Integerizer
from collections import Counter

## Using lark as a front end

In [2]:
grammar2 = r"""
WS: /[ \t\f\r\n]/
STAR: "*"
NUMBER: /\d+/

start: WS "SELECT" WS select_expr WS "FROM" WS from_expr
  [WS "WHERE" WS bool_condition] [WS "GROUP BY" WS var_list] [WS "ORDER BY" WS orderby_expr] WS EOS
EOS: "</s>"
select_expr: STAR | select_list
bool_condition: bool_expr | "(" bool_condition WS "AND" WS bool_condition ")" | "(" bool_condition WS "OR" WS bool_condition ")" 
bool_expr: var "=" value | var ">" value | var "<" value
from_expr: "data"
orderby_expr: var_list WS "ASC" | var_list WS "DESC"
select_list: select_var ("," WS select_var)*
var_list: var ("," WS var)*
select_var: var | "AVG(" var ")" | "MEDIAN(" var ")" | "COUNT(" var ")"
var: "age" | "gender" | "year" | "state_color" | "zipcode" | "vote" | "race_ethnicity"
value: NUMBER | "red" | "blue" | "white" | "black" | "latino" | "republican" | "democrat" | "male" | "female"

"""

In [3]:
grammar1 = """
start: query_expr EOS

EOS: "</s>"

query_expr: select [ "ORDER" "BY" (order_by_expr ",")*  order_by_expr] [ "LIMIT" integer_ ] 

select: "SELECT" [(select_expr ",")*] select_expr "FROM" "data" [ "WHERE" bool_expression ] [ "GROUP" "BY" [(expression ",")*] expression ]

select_expr.0: expression_math [ [ "AS" ] alias ] -> select_expression

?expression_math: expression_product
               | expression_math PLUS expression_product -> expression_add
               | expression_math "-" expression_product -> expression_sub
               | AGGREGATION expression_math /\)/ -> sql_aggregation

?expression: (name | STAR) -> column_name
            | literal

?expression_product: expression_parens
                  | expression_product STAR expression_parens
                  | expression_product "/" expression_parens 

?expression_parens: expression
                  | /\(/ expression_parens STAR expression /\)/ 
                  | /\(/  expression_parens "/" expression /\)/ 
                  | /\(/  expression_parens PLUS expression /\)/
                  | /\(/  expression_parens "-" expression /\)/

bool_expression: bool_parentheses
                 | bool_expression "AND" bool_parentheses 
                 | bool_expression "OR" bool_parentheses
bool_parentheses: comparison_type
                 | /\(/   bool_expression "AND" comparison_type /\)/
                 | /\(/  bool_expression "OR" comparison_type /\)/
comparison_type: equals | not_equals | greater_than | less_than | greater_than_or_equal
| less_than_or_equal | is_null | is_not_null
equals: expression_math "=" expression_math
not_equals: expression_math ("<>" | "!=") expression_math
greater_than: expression_math ">" expression_math
less_than: expression_math "<" expression_math
greater_than_or_equal: expression_math ">=" expression_math
less_than_or_equal: expression_math "<=" expression_math
is_null: expression_math "is" "null"
is_not_null: expression_math "is" "not" "null"

alias: /[A-Za-z]+/
name: /[A-Za-z]+/
PLUS: /\+/

order_by_expr: expression_math ["ASC"] -> order_asc
        | expression_math "DESC" -> order_desc

AGGREGATION.8: ("sum(" | "avg(" | "min(" | "max(" | "count(" "distinct" | "count(")
STAR: /\*/
integer_: /[1-9][0-9]*/
?literal: boolean -> bool
       | integer_ -> number
       | ESCAPED_STRING -> string

boolean: "true" -> true
       | "false" -> false

%import common.WS
%ignore WS
%import common.ESCAPED_STRING
    
"""

The following code is adapted from partenon.

In [4]:
raw_grammar = grammar1

lark_stuff = LarkStuff(raw_grammar)

In [5]:
intern = Integerizer()   # rename nonterminals to integers
g = lark_stuff.convert()
g = g.rename(intern)
assert g.in_cnf()    # lark returns a grammar in CNF
#g = g.cnf

In [6]:
g

0.058823529411764705: 1 → FALSE
1.0: 2 → 3 4
0.07692307692307693: 5 → 5 6
0.058823529411764705: 7 → 8 9
0.058823529411764705: 10 → 11 12
0.08333333333333333: 13 → 14 15
1.0: 16 → 17 18
1.0: 19 → 20 1
0.07692307692307693: 5 → 11 21
0.058823529411764705: 10 → 11 22
1.0: 23 → 24 25
1.0: 26 → 27 28
0.07692307692307693: 5 → 11 29
0.058823529411764705: 10 → TRUE
0.5: 27 → 10 30
1.0: 17 → FROM
1.0: 31 → LIMIT
1.0: 32 → OR
0.058823529411764705: 7 → 8 33
0.1: 34 → 14 35
1.0: 0 → 7 36
0.06666666666666667: 11 → 11 37
0.1111111111111111: 38 → 11 39
0.5: 40 → 11 41
1.0: 42 → 43 1
0.09090909090909091: 44 → 11 39
0.06666666666666667: 11 → __ANON_6
0.058823529411764705: 10 → STAR
0.08333333333333333: 13 → 14 35
1.0: 45 → 20 46
1.0: 14 → __ANON_1
1.0: 47 → IS
0.08333333333333333: 13 → 13 48
1.0: 49 → 50 3
0.07692307692307693: 5 → 11 41
1.0: 51 → 34 52
1.0: 53 → 10 54
0.058823529411764705: 10 → 14 15
1.0: 55 → 34 56
1.0: 57 → 24 58
1.0: 15 → 34 59
0.08333333333333333: 60 → 8 61
1.0: 62 → 63 64
1.0: 65 → 17 66
0.058823529411764705: 1 → 13 67
1.0: 56 → 68 69
1.0: 70 → 20 42
0.5: 40 → 11 71
0.058823529411764705: 1 → 13 48
0.06666666666666667: 11 → TRUE
1.0: 72 → 5 73
1.0: 74 → 75 76
1.0: 77 → LESSTHAN
1.0: 61 → 10 16
1.0: 78 → PLUS
1.0: 79 → 75 80
0.16666666666666666: 3 → __ANON_7
1.0: 81 → AND
1.0: 82 → 24 83
1.0: 66 → 75 84
1.0: 85 → 50 3
1.0: 86 → 68 13
1.0: 87 → 10 88
0.058823529411764705: 7 → 8 61
0.16666666666666666: 3 → ESCAPED_STRING
0.1111111111111111: 38 → 11 71
0.09090909090909091: 44 → 11 71
0.06666666666666667: 11 → STAR
0.058823529411764705: 10 → 14 35
0.1: 34 → FALSE
1.0: 89 → 75 90
1.0: 91 → 31 92
0.1111111111111111: 38 → 11 93
0.09090909090909091: 44 → 11 93
1.0: 94 → 38 4
0.08333333333333333: 13 → FALSE
0.07692307692307693: 5 → 14 95
0.058823529411764705: 1 → 14 55
1.0: 30 → COMMA
1.0: 29 → 47 96
1.0: 97 → 11 21
1.0: 25 → 5 98
0.058823529411764705: 1 → 11 99
1.0: 100 → NOT
0.1111111111111111: 38 → 11 101
0.1111111111111111: 38 → 11 102
0.09090909090909091: 44 → 11 101
0.09090909090909091: 44 → 11 102
1.0: 103 → 1 30
1.0: 104 → 63 45
1.0: 105 → __ANON_3
1.0: 20 → BY
1.0: 22 → 106 12
1.0: 107 → 11 102
0.08333333333333333: 60 → 8 108
0.08333333333333333: 60 → 8 33
1.0: 109 → 1 110
1.0: 76 → 63 111
1.0: 112 → ORDER
1.0: 113 → 27 114
1.0: 73 → 32 94
1.0: 18 → 75 115
0.058823529411764705: 1 → 11 86
1.0: 116 → 75 117
1.0: 118 → 27 53
0.5: 119 → TRUE
0.08333333333333333: 60 → 8 120
0.058823529411764705: 1 → 14 51
0.07692307692307693: 5 → 14 72
0.25: 121 → __ANON_7
0.06666666666666667: 11 → 14 35
1.0: 122 → 32 44
0.058823529411764705: 1 → 123 124
0.1111111111111111: 38 → 11 21
0.058823529411764705: 7 → 60 125
0.16666666666666666: 3 → __ANON_6
0.09090909090909091: 44 → 11 21
0.06666666666666667: 11 → 13 48
1.0: 126 → 20 85
0.1111111111111111: 38 → 11 29
1.0: 21 → 127 11
0.25: 121 → ESCAPED_STRING
0.09090909090909091: 44 → 11 29
1.0: 80 → 24 5
0.058823529411764705: 10 → FALSE
1.0: 90 → 63 128
0.058823529411764705: 7 → 8 108
1.0: 129 → 20 3
1.0: 130 → 75 23
0.058823529411764705: 7 → 8 120
0.058823529411764705: 1 → __ANON_7
0.08333333333333333: 13 → 13 67
1.0: 92 → __ANON_7
1.0: 131 → 112 70
1.0: 132 → STAR
1.0: 133 → 75 104
0.058823529411764705: 1 → ESCAPED_STRING
1.0: 134 → 75 57
0.1111111111111111: 38 → 11 41
1.0: 135 → SLASH
1.0: 136 → 27 137
0.09090909090909091: 44 → 11 41
0.16666666666666666: 3 → TRUE
1.0: 138 → 17 133
0.058823529411764705: 7 → 60 131
1.0: 139 → NULL
0.5: 50 → 50 140
0.058823529411764705: 7 → 60 141
1.0: 67 → 135 34
1.0: 142 → 17 74
0.16666666666666666: 3 → STAR
1.0: 124 → 11 4
1.0: 9 → 27 87
0.06666666666666667: 11 → FALSE
1.0: 143 → 1 144
1.0: 98 → 63 126
0.1: 34 → 14 55
1.0: 59 → 78 2
1.0: 145 → 17 75
1.0: 6 → 81 44
0.07692307692307693: 5 → 11 146
0.06666666666666667: 11 → 11 86
1.0: 147 → 20 143
1.0: 35 → 34 148
1.0: 115 → 24 149
1.0: 102 → 77 11
1.0: 64 → 20 3
0.058823529411764705: 10 → 13 67
0.08333333333333333: 13 → 14 55
1.0: 83 → 5 150
0.06666666666666667: 11 → 123 124
1.0: 151 → 17 130
1.0: 46 → 50 3
0.0588235

In [7]:
len(g.rules), len(g.V), len(g.N)

(320, 38, 184)

In [8]:
sorted(g.cnf.V)

['AGGREGATION',
 'AND',
 'AS',
 'ASC',
 'BY',
 'COMMA',
 'DATA',
 'DESC',
 'EOS',
 'EQUAL',
 'ESCAPED_STRING',
 'FALSE',
 'FROM',
 'GROUP',
 'IS',
 'LESSTHAN',
 'LIMIT',
 'MINUS',
 'MORETHAN',
 'NOT',
 'NULL',
 'OR',
 'ORDER',
 'PLUS',
 'SELECT',
 'SLASH',
 'STAR',
 'TRUE',
 'WHERE',
 'WS',
 '__ANON_0',
 '__ANON_1',
 '__ANON_2',
 '__ANON_3',
 '__ANON_4',
 '__ANON_5',
 '__ANON_6',
 '__ANON_7']

In [9]:
#from newton.linking import LinkAnalysis
#f = Integerizer()
#links = LinkAnalysis(g.rename(f))
#links.dfs

In [10]:
g.language(6)

Counter({('SELECT', 'TRUE', 'FROM', 'DATA', 'EOS'): 0.0034602076124567475,
         ('SELECT', 'STAR', 'FROM', 'DATA', 'EOS'): 0.0034602076124567475,
         ('SELECT', 'FALSE', 'FROM', 'DATA', 'EOS'): 0.0034602076124567475,
         ('SELECT', '__ANON_7', 'FROM', 'DATA', 'EOS'): 0.0034602076124567475,
         ('SELECT',
          'ESCAPED_STRING',
          'FROM',
          'DATA',
          'EOS'): 0.0034602076124567475,
         ('SELECT', '__ANON_6', 'FROM', 'DATA', 'EOS'): 0.0034602076124567475,
         ('SELECT',
          'TRUE',
          'FROM',
          'DATA',
          'LIMIT',
          '__ANON_7',
          'EOS'): 0.0002883506343713956,
         ('SELECT',
          'STAR',
          'FROM',
          'DATA',
          'LIMIT',
          '__ANON_7',
          'EOS'): 0.0002883506343713956,
         ('SELECT',
          'FALSE',
          'FROM',
          'DATA',
          'LIMIT',
          '__ANON_7',
          'EOS'): 0.0002883506343713956,
         ('SELECT',
  

## Tokenization

We can extract lark's tokenizer in a format that we can build on.  We will even make a DIY tokenizer based on Python's `re` library.

| Terminology  |         |
|--------------|---------|
| tokenization | lexing  |
| tokenizers   | lexers  |
| tokens       | lexemes |



In [11]:
sorted(lark_stuff.all_terminals, key=lambda t: -t.priority)

[TerminalDef('AGGREGATION', '(?:count\\(distinct|count\\(|sum\\(|avg\\(|min\\(|max\\()'),
 TerminalDef('ESCAPED_STRING', '".*?(?<!\\\\)(\\\\\\\\)*?"'),
 TerminalDef('WS', '(?:[ \t\x0c\r\n])+'),
 TerminalDef('EOS', '</s>'),
 TerminalDef('PLUS', '\\+'),
 TerminalDef('STAR', '\\*'),
 TerminalDef('COMMA', ','),
 TerminalDef('ORDER', 'ORDER'),
 TerminalDef('BY', 'BY'),
 TerminalDef('LIMIT', 'LIMIT'),
 TerminalDef('WHERE', 'WHERE'),
 TerminalDef('GROUP', 'GROUP'),
 TerminalDef('SELECT', 'SELECT'),
 TerminalDef('FROM', 'FROM'),
 TerminalDef('DATA', 'data'),
 TerminalDef('AS', 'AS'),
 TerminalDef('MINUS', '\\-'),
 TerminalDef('__ANON_0', '\\)'),
 TerminalDef('SLASH', '/'),
 TerminalDef('__ANON_1', '\\('),
 TerminalDef('AND', 'AND'),
 TerminalDef('OR', 'OR'),
 TerminalDef('EQUAL', '='),
 TerminalDef('__ANON_2', '<>'),
 TerminalDef('__ANON_3', '!='),
 TerminalDef('MORETHAN', '>'),
 TerminalDef('LESSTHAN', '<'),
 TerminalDef('__ANON_4', '>='),
 TerminalDef('__ANON_5', '<='),
 TerminalDef('IS', 'i

### DIY tokenizer

In [12]:
text = "12 + 24 - 36 * 48 / 60 SELECT table.name AS thing WHERE table.potato IS NOT 'banana'"

for x, y in lark_stuff.simple_tokenizer(text):
    print(f'{x:15s} -> {y!r}')

__ANON_7        -> '12'
PLUS            -> '+'
__ANON_7        -> '24'
MINUS           -> '-'
__ANON_7        -> '36'
STAR            -> '*'
__ANON_7        -> '48'
SLASH           -> '/'
__ANON_7        -> '60'
SELECT          -> 'SELECT'
__ANON_6        -> 'table'
__ANON_6        -> 'name'
AS              -> 'AS'
__ANON_6        -> 'thing'
WHERE           -> 'WHERE'
__ANON_6        -> 'table'
__ANON_6        -> 'potato'
__ANON_6        -> 'IS'
__ANON_6        -> 'NOT'
__ANON_6        -> 'banana'


### Parsing tokenized input

In [13]:
text = 'SELECT name FROM data </s>'

In [14]:
tokens = list(lark_stuff.lex(text))
tokens

[Token('SELECT', 'SELECT'),
 Token('__ANON_6', 'name'),
 Token('FROM', 'FROM'),
 Token('DATA', 'data'),
 Token('EOS', '</s>')]

Call the lark parser on the text:

In [15]:
lark_stuff.instance.parse(text)

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'query_expr'), [Tree(Token('RULE', 'select'), [Tree('select_expression', [Tree('column_name', [Tree(Token('RULE', 'name'), [Token('__ANON_6', 'name')])]), None]), None, None]), None, None]), Token('EOS', '</s>')])

We can call the lark parser on these tokens:

In [16]:
lark_stuff.parser.parse(tokens, 'start')

Tree(NonTerminal(Token('RULE', 'start')), [Tree(NonTerminal(Token('RULE', 'query_expr')), [Tree(NonTerminal(Token('RULE', 'select')), [Token('SELECT', 'SELECT'), Tree(NonTerminal(Token('RULE', 'select_expr')), [Tree(NonTerminal(Token('RULE', 'expression_math')), [Tree(NonTerminal(Token('RULE', 'expression_product')), [Tree(NonTerminal(Token('RULE', 'expression_parens')), [Tree(NonTerminal(Token('RULE', 'expression')), [Tree(NonTerminal(Token('RULE', 'name')), [Token('__ANON_6', 'name')])])])])])]), Token('FROM', 'FROM'), Token('DATA', 'data')])]), Token('EOS', '</s>')])

We can call our parser on this text to get its total weight

In [17]:
g([t.type for t in tokens])

0.0034602076124567475

### Tokenizer State Machines

**TODO**: Dive into the [greenery](https://github.com/qntm/greenery) FSMs to figure out how they work; we can work backward from the Partenon tensor building routine.  Another option might be the [interegular](https://github.com/MegaIng/interegular) library.


**Note**: Tokenizers are FSTs, not FSAs.  However, these libraries implement the kind of restricted FSTs with a separate FSA per token type.

In [18]:
import greenery

def make_greenery_fsms(
    regex_list, 
    ignore = ("\s*",), 
    chars = tuple(chr(i) for i in range(128))
):
    match ignore:
        case []:
            ignore_regex = ""
        case [ignore]:
            ignore_regex = ignore
        case _:
            raise ValueError("ignore must be a list of length at most 1")

    patterns = []
    for regex in regex_list:
        # greenery does not escape spaces
        regex = regex.replace("\\ ", " ")
        patterns.append(greenery.parse(regex + ignore_regex))

    return [pattern.to_fsm() for pattern in patterns]

In [19]:
fsms = make_greenery_fsms([t for t in tokens])

In [20]:
fsms[0]

Fsm(alphabet=frozenset({Charclass((('E', 'E'),)), Charclass((('T', 'T'),)), Charclass((('!', 'B'), ('D', 'D'), ('F', 'K'), ('M', 'R'))), ~Charclass((('\t', 'T'),)), Charclass((('L', 'L'),)), Charclass((('\t', '\r'), (' ', ' '))), Charclass((('\x0e', '\x1f'),)), Charclass((('S', 'S'),)), Charclass((('C', 'C'),))}), states=frozenset({0, 1, 2, 3, 4, 5, 6, 7}), initial=0, finals=frozenset({7}), map={0: {Charclass((('\t', '\r'), (' ', ' '))): 1, Charclass((('\x0e', '\x1f'),)): 1, Charclass((('!', 'B'), ('D', 'D'), ('F', 'K'), ('M', 'R'))): 1, Charclass((('C', 'C'),)): 1, Charclass((('E', 'E'),)): 1, Charclass((('L', 'L'),)): 1, Charclass((('S', 'S'),)): 2, Charclass((('T', 'T'),)): 1, ~Charclass((('\t', 'T'),)): 1}, 1: {Charclass((('\t', '\r'), (' ', ' '))): 1, Charclass((('\x0e', '\x1f'),)): 1, Charclass((('!', 'B'), ('D', 'D'), ('F', 'K'), ('M', 'R'))): 1, Charclass((('C', 'C'),)): 1, Charclass((('E', 'E'),)): 1, Charclass((('L', 'L'),)): 1, Charclass((('S', 'S'),)): 1, Charclass((('T', '

The code below was used in [partenon](https://github.com/probcomp/partenon) to convert the the lark tokenizer `->` greenery FSMs `->` tensors.  We can repurpose it to convert into a transducer from characters to token names.

In [21]:
def make_matrix_from_fsms(fsms, chars):
    max_states = max([len(fsm.states) for fsm in fsms])
    n_inputs = len(chars)

    m_shape = (len(fsms), max_states, n_inputs, max_states)
    m = np.zeros(m_shape, dtype=np.int8)
    finals = jnp.zeros((len(fsms), max_states))
    for fsm_idx, fsm in enumerate(fsms):
        for final_state in fsm.finals:
            finals = finals.at[fsm_idx, final_state].set(1)
        fsm_states = fsm.states
        rejection_states = [e for e in fsm_states if not fsm.islive(e)]
        for state in fsm_states:
            arcs = fsm.map[state]
            for input_char, next_state in arcs.items():
                if next_state in rejection_states:  # rejection state
                    continue
                for char in input_char.get_chars():
                    input_idx = chars.index(char)
                    m[fsm_idx, state, input_idx, next_state] = 1

    m = jnp.array(m)

    return m, finals, max_states

## Prefix Grammar

In [22]:
len(g.cnf.rules), len(g.cnf.prefix_grammar.trim().rules), len(g.cnf.prefix_grammar.trim().rules)/len(g.cnf.rules)

(304, 1220, 4.0131578947368425)

In [23]:
#len(g.cnf.prefix_grammar.nullaryremove().trim().rules) / len(g.cnf.rules)

In [24]:
#from dyna.util import Graph
#def unary_graph(self):
#    return Graph([(p.body[0], p.head) for p in self if len(p.body) == 1 and self.is_nonterminal(p.body[0])])

G = g.cnf.prefix_grammar.trim()

#print(G.num_rules, len(G.N), len(G.V))
#U = unary_graph(G)
#C = U.condensation()
#print('largest scc:', max(map(len, C.nodes), default=0))

In [25]:
#C

In [26]:
N = G.nullaryremove()    # could be faster with SCC-based prioritization

In [ ]:
N.unaryremove()   # currently, very solve because we dont have SCC-decomposed Lehmann's algorithm

In [ ]:
g.cnf.prefix_grammar.trim().cnf

In [ ]:
g.prefix_weight([t.type for t in tokens])